In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import multiprocessing
import numpy as np
import shutil 
import preprocessor as p
import pickle
from textblob import TextBlob
import itertools 

%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 500

In [3]:
%run src/helper_functions.py

In [ ]:
%run src/cities_sentiment.py

### Read in twitter and cdc data

In [5]:
# read in twitter data as chunks
df = pd.read_json('data/twitter/output_file_weekend.txt', lines = True)

In [ ]:
# read in CDC data
cities = pd.read_csv('data/cdc/500_Cities__City-level_Data__GIS_Friendly_Format___2018_release.csv')

### process data

In [ ]:
clean_one(df)
clean_text(df_clean)
sentiment(df_clean)
map_city(df_clean)
cdc_cut(cities)

In [11]:
#merge to cities CDC data
prediction6 = pd.merge(prediction5,cities,  
                       left_on=('closest_city','closest_state'), 
                       right_on=('PlaceName','StateAbbr'), 
                       indicator = True)

prediction6.drop_duplicates(inplace=True)

# output data to pickle
pd.to_pickle(prediction6, 'data/cities_sentiment_data.pkl')

### checks

In [166]:
#look at the count by bin
count_by_bin = prediction_cut2.groupby(['OBESITY_cut','sentiment'])['polarity'].agg(['count','mean']).reset_index()
count_by_bin.pivot_table(index='OBESITY_cut', columns="sentiment", values=('mean', 'count'))

count                       mean                  
sentiment   negative neutral positive  negative neutral  positive
OBESITY_cut                                                      
low              127     386      484 -0.273935     0.0  0.414024
medium           155     385      540 -0.266884     0.0  0.411987
high             161     328      675 -0.296340     0.0  0.459977

In [177]:
#look at the count by city
count_by_city = prediction.groupby(['closest_major_city','sentiment'])['polarity'].agg(['count','mean']).reset_index()
count_by_city.pivot_table(index='closest_major_city', columns="sentiment", values=('mean', 'count'))

count                       mean                  
sentiment            negative neutral positive  negative neutral  positive
closest_major_city                                                        
Abilene, TX               NaN     1.0      2.0       NaN     0.0  0.280000
Akron, OH                 2.0     6.0      3.0 -0.440625     0.0  0.351667
Albuquerque, NM           1.0     4.0      2.0 -0.187500     0.0  0.687500
Alexandria, VA            5.0    22.0     15.0 -0.155000     0.0  0.345094
Alhambra, CA              2.0     4.0      1.0 -0.150000     0.0  0.600000
Allentown, PA             1.0     4.0      7.0 -0.210000     0.0  0.360763
Amarillo, TX              NaN     4.0      8.0       NaN     0.0  0.538750
Anaheim, CA               NaN     NaN      2.0       NaN     NaN  0.900000
Anchorage, AK             2.0     3.0      1.0 -0.177778     0.0  0.850000
Ann Arbor, MI             2.0     5.0      7.0 -0.462500     0.0  0.441964
Antioch, CA               NaN     1.0      1.0       NaN     0.0  0.483333
Arlington, TX             2.0     1.0      2.0 -0.668750     0.0  0.325000
Arlington, VA             NaN    11.0     15.0       NaN     0.0  0.327961
Arvada, CO                1.0     1.0      1.0 -0.200000     0.0  0.425000
Athens, GA                NaN     2.0      3.0       NaN     0.0  0.212121
Atlanta, GA               4.0    22.0     32.0 -0.272049     0.0  0.404178
Augusta, GA               1.0     NaN      2.0 -0.750000     NaN  0.328906
Austin, TX                9.0    24.0     44.0 -0.225926     0.0  0.442239
Avondale, AZ              6.0     8.0     11.0 -0.350810     0.0  0.406315
Bakersfield, CA           NaN     NaN      2.0       NaN     NaN  0.600000
Baltimore, MD             7.0    19.0     27.0 -0.373810     0.0  0.554541
Baton Rouge, LA           2.0     1.0      2.0 -0.143750     0.0  0.600000
Beaumont, TX              1.0     2.0      NaN -0.364323     0.0       NaN
Beaverton, OR             4.0     6.0     12.0 -0.603125     0.0  0.337734
Bellevue, WA              1.0     3.0      NaN -0.587500     0.0       NaN
Bellingham, WA            NaN     NaN      3.0       NaN     NaN  0.313889
Bend, OR                  NaN     NaN      1.0       NaN     NaN  1.000000
Berkeley, CA              NaN     2.0      2.0       NaN     0.0  0.450000
Billings, MT              NaN     1.0      NaN       NaN     0.0       NaN
Birmingham, AL            1.0     1.0      1.0 -0.976562     0.0  0.112500
Bloomington, MN           2.0     7.0     11.0 -0.346429     0.0  0.324289
Boca Raton, FL            NaN     1.0      2.0       NaN     0.0  0.391667
Boston, MA                NaN     NaN      1.0       NaN     NaN  1.000000
Boulder, CO               2.0     3.0      4.0 -0.475000     0.0  0.385448
Bridgeport, CT            NaN     1.0      2.0       NaN     0.0  0.197619
Brockton, MA              NaN     NaN      4.0       NaN     NaN  0.387500
Broken Arrow, OK          NaN     3.0     10.0       NaN     0.0  0.427195
Brownsville, TX           1.0     2.0      4.0 -0.133333     0.0  0.201042
Buena Park, CA            NaN     1.0      2.0       NaN     0.0  0.245000
Buffalo, NY               3.0     2.0      5.0 -0.397222     0.0  0.268167
Burbank, CA               NaN     3.0      3.0       NaN     0.0  0.433333
Cambridge, MA             NaN     NaN      1.0       NaN     NaN  0.375000
Camden, NJ                3.0    11.0     17.0 -0.106085     0.0  0.414451
Cape Coral, FL            NaN     1.0      1.0       NaN     0.0  0.350000
Carlsbad, CA              2.0     1.0      4.0 -0.041250     0.0  0.553125
Carson, CA                NaN     3.0      3.0       NaN     0.0  0.128704
Cary, NC                  1.0     5.0      9.0 -0.170000     0.0  0.558025
Cedar Rapids, IA          NaN     2.0      4.0       NaN     0.0  0.435937
Centennial, CO            NaN     NaN      3.0       NaN     NaN  0.611111
Champaign, IL             NaN     1.0      NaN       NaN     0.0       NaN
Chandler, AZ              3.0     5.0      6.0 -0

In [183]:
len(prediction6)

327